In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download paultimothymooney/chest-xray-pneumonia
! unzip chest-xray-pneumonia.zip

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import cv2 
from skimage import io


from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, ConfusionMatrixDisplay

In [4]:
train_datagen = ImageDataGenerator(
        rescale= 1./255, # uint(unassigned integer) -> float by 1.
        rotation_range = 20,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        fill_mode='constant')

In [5]:
train_generator = train_datagen.flow_from_directory('/content/chest_xray/train',
                                                    target_size=(150,150),
                                                    class_mode = 'sparse',
                                                    batch_size = 32,
                                                    color_mode = 'grayscale')

Found 5216 images belonging to 2 classes.


In [6]:
train_generator.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [7]:
valid_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
validation_generator = valid_datagen.flow_from_directory('/content/chest_xray/test', 
                                                          target_size=(150,150),
                                                          class_mode = 'sparse',
                                                          batch_size = 32,
                                                          color_mode = 'grayscale'
                                                          )

Found 624 images belonging to 2 classes.


In [9]:
validation_generator.class_indices

{'NORMAL': 0, 'PNEUMONIA': 1}

In [15]:
dense_layers = [2] 
layer_sizes = [128]
conv_layers = [1]

for dense_layer in dense_layers:
  for layer_size in layer_sizes:
    for conv_layer in conv_layers:

      # Create a model
      model = Sequential()

      # Convolutional Layers
      model.add(Conv2D(layer_size, (3, 3), input_shape=(150,150,1), activation = 'relu'))
      model.add(MaxPooling2D(pool_size=(2,2)))

      for _ in range(conv_layer-1):
        model.add(Conv2D(layer_size, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2,2)))
      
      # Flatten Layer
      model.add(Flatten())
      
      # Dense Layers
      for _ in range(dense_layer):
        model.add(Dense(layer_size, activation='relu'))

      # Output Layer
      model.add(Dense(2, activation='softmax'))

      model.compile(loss='sparse_categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
      model.summary()

      early_stopping_cb = EarlyStopping(patience=3, monitor = 'val_loss', restore_best_weights=True)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 148, 148, 128)     1280      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 74, 74, 128)      0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 700928)            0         
                                                                 
 dense_3 (Dense)             (None, 128)               89718912  
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 2)                 258       
                                                      

In [16]:
# Data_augmented_fitting
model.fit(train_generator,
          steps_per_epoch = 5216 // 32, # // : return round value
          epochs = 10,
          validation_data = validation_generator,
          validation_steps = 624 // 32,
          callbacks=[early_stopping_cb],
          verbose = 1)

Epoch 1/10
163/163 [==============================] - 74s 448ms/step - loss: 0.6136 - accuracy: 0.7450 - val_loss: 0.5519 - val_accuracy: 0.7812
Epoch 2/10
163/163 [==============================] - 72s 440ms/step - loss: 0.3696 - accuracy: 0.8349 - val_loss: 0.4161 - val_accuracy: 0.8372
Epoch 3/10
163/163 [==============================] - 69s 424ms/step - loss: 0.3269 - accuracy: 0.8520 - val_loss: 0.4238 - val_accuracy: 0.8322
Epoch 4/10
163/163 [==============================] - 71s 434ms/step - loss: 0.3113 - accuracy: 0.8620 - val_loss: 0.4579 - val_accuracy: 0.8273
Epoch 5/10
163/163 [==============================] - 72s 442ms/step - loss: 0.2905 - accuracy: 0.8725 - val_loss: 0.3917 - val_accuracy: 0.8438
Epoch 6/10
163/163 [==============================] - 68s 416ms/step - loss: 0.2701 - accuracy: 0.8771 - val_loss: 0.4219 - val_accuracy: 0.8586
Epoch 7/10
163/163 [==============================] - 69s 424ms/step - loss: 0.2860 - accuracy: 0.8748 - val_loss: 0.3966 - val_ac

In [17]:
model.save('aug_model.h5')